# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [40]:
import numpy as np
import pandas as pd

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [41]:
dtype = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype = dtype)
train = pd.read_csv('kc_house_train_data.csv', dtype = dtype)
test = pd.read_csv('kc_house_test_data.csv', dtype = dtype)

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [42]:
def get_numpy_data(
        # data_sframe, 
        data_frame, features, output):
    # data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    data_frame = data_frame.assign(constant = 1)
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    feature_df = data_frame.loc[:, features]
    # the following line will convert the features_SFrame into a numpy matrix:
    # feature_matrix = features_sframe.to_numpy()
    feature_matrix = feature_df.values
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_df = data_frame.loc[:, output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    # output_array = output_sarray.to_numpy()
    output_array = output_df.values
    
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [43]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [44]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print(X)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [45]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print(norms)

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [46]:
print(X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [47]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis = 0)
    normalized_features = feature_matrix / norms
    
    return (normalized_features, norms)

To test the function, run the following:

In [48]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print(features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print(norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [49]:
feature_matrix, output_array = get_numpy_data(sales, ['sqft_living', 'bedrooms'], ['price'])

Don't forget to normalize features:

In [50]:
normalized_features, norms = normalize_features(feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [51]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [52]:
predictions = predict_output(feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [53]:
# compute value of ro[i]
ro = []

ro.append(np.sum(normalized_features[:, 0] * (output_array[:, 0] - predictions + weights[0] * normalized_features[:, 0])))
ro.append(np.sum(normalized_features[:, 1] * (output_array[:, 0] - predictions + weights[1] * normalized_features[:, 1])))
ro.append(np.sum(normalized_features[:, 2] * (output_array[:, 0] - predictions + weights[2] * normalized_features[:, 2])))


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [54]:
from decimal import Decimal

print('{:.2E}'.format(ro[0]))
print('{:.2E}'.format(ro[1]))
print('{:.2E}'.format(ro[2]))

7.82E+07
8.66E+07
7.97E+07


In [55]:
l1_penalty_min_qz1 = 2 * ro[2]
l1_penalty_max_qz1 = 2 * ro[1]  # exclusive

print('range is [{:.2E}, {:.3E})'.format(l1_penalty_min_qz1, l1_penalty_max_qz1))

range is [1.59E+08, 1.732E+08)


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [56]:
l1_penalty_min_qz2 = 2 * ro[1]

print('range is [{:.3E}, infinity)'.format((l1_penalty_min_qz2)))

range is [1.732E+08, infinity)


So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [57]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.sum(feature_matrix[:, i] * (output - prediction + weights[i] * feature_matrix[:, i]))

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [58]:
# should print 0.425558846691
import math
print(lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],
                   [2./math.sqrt(13),3./math.sqrt(10)]]), np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [59]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights
    len_w = len(weights)
    j = 1
    while True:
        max_delta = 0
        for i in range(len_w):
            old_weight = weights[i]
            new_weight = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            weights[i] = new_weight
            delta_i = abs(new_weight - old_weight)
            if delta_i > max_delta:
                max_delta = delta_i
        if j % 10 == 0 and j // 10 > 0:
            print('round {}, max_delta is {:.2f}'.format(j, max_delta))
        if max_delta < tolerance:
            break
        j = j + 1
    return weights

Using the following parameters, learn the weights on the sales dataset. 

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

Then, run your implementation of LASSO coordinate descent:

In [60]:
feature_matrix, output_array = get_numpy_data(sales, ['sqft_living', 'bedrooms'], ['price'])
normalized_features, norms = normalize_features(feature_matrix)

weights_qz3 = lasso_cyclical_coordinate_descent(
    normalized_features, 
    output_array[:, 0], 
    np.array([0, 0, 0]), 
    1e+7, 
    1.0)

round 10, max_delta is 2266212.32
round 20, max_delta is 381663.88
round 30, max_delta is 64278.04
round 40, max_delta is 10825.01
round 50, max_delta is 1822.21
round 60, max_delta is 306.88
round 70, max_delta is 50.94
round 80, max_delta is 8.49
round 90, max_delta is 1.43


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [61]:
err_qz3 = predict_output(normalized_features, weights_qz3) - output_array[:, 0]
rss_qz3 = np.dot(err_qz3, err_qz3)
print('RSS is %.2E' % rss_qz3)

RSS is 1.63E+15


# Evaluating LASSO fit with more features

Let us consider the following set of features.

In [62]:
all_features_qz4 = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']



First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [63]:
feature_matrix, output_array = get_numpy_data(
    train.astype({'floors': 'float64'}), 
    all_features_qz4, ['price'])

normalized_features, norms = normalize_features(feature_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [64]:
weights1e7 = lasso_cyclical_coordinate_descent(
    normalized_features, 
    output_array[:, 0], 
    np.repeat(0, len(all_features_qz4)+1), 
    l1_penalty = 1e+7, 
    tolerance = 1)

round 10, max_delta is 2706151.13
round 20, max_delta is 222753.24
round 30, max_delta is 40345.69
round 40, max_delta is 7307.62
round 50, max_delta is 1323.31
round 60, max_delta is 239.10
round 70, max_delta is 43.29
round 80, max_delta is 8.38
round 90, max_delta is 2.22


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [65]:
np_features = np.array(['constant']+all_features_qz4)
np_features[weights1e7 != 0]

array(['constant', 'sqft_living', 'waterfront', 'view'], dtype='<U13')

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [66]:
weights1e8 = lasso_cyclical_coordinate_descent(
    normalized_features, 
    output_array[:, 0], 
    np.repeat(0, len(all_features_qz4)+1), 
    l1_penalty = 1e+8, 
    tolerance = 1)

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [67]:
np_features[weights1e8 != 0]

array(['constant'], dtype='<U13')

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [68]:
weights1e4 = lasso_cyclical_coordinate_descent(
    normalized_features, 
    output_array[:, 0], 
    np.repeat(0, len(all_features_qz4)+1), 
    l1_penalty = 1e+4, 
    tolerance = 5e+5)

round 10, max_delta is 1647982.54
round 20, max_delta is 1177722.10
round 30, max_delta is 1092904.48
round 40, max_delta is 965783.06
round 50, max_delta is 832351.92
round 60, max_delta is 717339.56
round 70, max_delta is 623694.08
round 80, max_delta is 552427.87
round 90, max_delta is 497885.44


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [69]:
np_features[weights1e4 != 0]

array(['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated'], dtype='<U13')

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [70]:
normalized_weights1e4 = weights1e4 / norms
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [71]:
print(normalized_weights1e7[3])

161.31746840896935


## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [77]:
(test_feature_matrix, test_output) = get_numpy_data(test.astype({'floors': 'float64'}), 
    all_features_qz4, ['price'])

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [79]:
err_1e4 = predict_output(test_feature_matrix, normalized_weights1e4) - test_output[:, 0]
rss_1e4 = np.dot(err_1e4, err_1e4)
err_1e7 = predict_output(test_feature_matrix, normalized_weights1e7) - test_output[:, 0]
rss_1e7 = np.dot(err_1e7, err_1e7)
err_1e8 = predict_output(test_feature_matrix, normalized_weights1e8) - test_output[:, 0]
rss_1e8 = np.dot(err_1e8, err_1e8)

***QUIZ QUESTION***

Which model performed best on the test data?

In [81]:
print('{:.2E}'.format(rss_1e4))
print('{:.2E}'.format(rss_1e7))
print('{:.2E}'.format(rss_1e8))

2.28E+14
2.76E+14
5.37E+14
